The purpose of this notebook is to explore running the input data to the model via the tf.data api instead of feed dicts as that is deemed more efficient to prevent GPU starvation, with possibility of optimizing the input pipeline by doing threading and prefatching.

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f2e11ccb208>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f2dce2cefd0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f2dc723b160>)

In [0]:
train_data = mnist.train

In [0]:
test_data = mnist.test

In [0]:
val_data = mnist.validation

In [0]:
# settting variables
image_height = 28
image_width = 28
image_pixel = 28 * 28
batch_size = 32

In [9]:
train_data.labels.shape

(55000, 10)

In [10]:
train_data.images.shape

(55000, 784)

In [0]:
# getting the number of data instance for each of the datasets
num_train_data = train_data.labels.shape[0]
num_test_data = test_data.labels.shape[0]
num_val_data = val_data.labels.shape[0]

In [0]:
temp = train_data.next_batch(100)

Going to try to use tf data as a way to feed into model

In [0]:
# we wanted unlimited looping so repeat indefinitely
dataset_train = tf.data.Dataset.from_tensor_slices((train_data.images, train_data.labels)).batch(32).repeat()

In [14]:
type(train_data.images)

numpy.ndarray

In [15]:
dataset_train

<RepeatDataset shapes: ((?, 784), (?, 10)), types: (tf.float32, tf.float64)>

In [0]:
# if I do not use batch, the iterator initialized from this dataset will do instance by instance iteration.
# This is a trick to not have to use another iterator. Basically batch_size = test data size so I only have 1 batch of everything.
dataset_test = tf.data.Dataset.from_tensor_slices((test_data.images, test_data.labels)).batch(num_test_data).repeat()

In [0]:
dataset_val = tf.data.Dataset.from_tensor_slices((val_data.images, val_data.labels)).batch(num_val_data).repeat()

In [0]:
itera = tf.data.Iterator.from_structure(dataset_train.output_types, dataset_train.output_shapes)

In [0]:
train_init_op = itera.make_initializer(dataset_train)
test_init_op = itera.make_initializer(dataset_test)
val_init_op = itera.make_initializer(dataset_val)

In [0]:
instance, label = itera.get_next()

In [21]:
train_data.images.shape

(55000, 784)

In [22]:
55000/32

1718.75

In [23]:
55000 - 32 * 1718

24

In [0]:
num_iterations_per_epoch = num_train_data // batch_size
if num_train_data % batch_size != 0:
  num_iterations_per_epoch += 1

In [25]:
num_iterations_per_epoch

1719

In [26]:
''' commented out this cell
first = None
last = None
second = None
test = None

with tf.Session() as sess:
  sess.run(train_init_op)
#   for _ in range(num_epoch):
#     print(sess.run(tf.shape(instance)))
  for i in range(3438):
    result = sess.run(instance)
    if i >= 3430 and i < 3445:
      print(result.shape)
#     print(i)
#     if i == 0:
#       first = sess.run(tf.Print(instance, [instance]))
#     if i == 1716:
# #       print(sess.run(tf.shape(instance)))
#       last = sess.run(tf.Print(instance, [instance]))
#     if i == 1716:
#       test = sess.run(tf.Print(instance, [instance]))
#     if i == 2:
#       second = sess.run(tf.Print(instance, [instance]))
'''

' commented out this cell\nfirst = None\nlast = None\nsecond = None\ntest = None\n\nwith tf.Session() as sess:\n  sess.run(train_init_op)\n#   for _ in range(num_epoch):\n#     print(sess.run(tf.shape(instance)))\n  for i in range(3438):\n    result = sess.run(instance)\n    if i >= 3430 and i < 3445:\n      print(result.shape)\n#     print(i)\n#     if i == 0:\n#       first = sess.run(tf.Print(instance, [instance]))\n#     if i == 1716:\n# #       print(sess.run(tf.shape(instance)))\n#       last = sess.run(tf.Print(instance, [instance]))\n#     if i == 1716:\n#       test = sess.run(tf.Print(instance, [instance]))\n#     if i == 2:\n#       second = sess.run(tf.Print(instance, [instance]))\n'

In [0]:
# write model
input_tensor = tf.reshape(instance, [-1, image_height, image_width, 1])

conv1 = tf.layers.conv2d(inputs=input_tensor, filters=32, kernel_size=[5, 5], activation='relu', padding='same', strides=[2, 2], name='conv_layer_1')

conv1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name='max_pool_1')

conv2 = tf.layers.conv2d(inputs=conv1, filters=64, kernel_size=[5, 5], activation='relu', padding='same', strides=1, name='conv_layer_2')

conv2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name='max_pool_2')

flat1 = tf.layers.flatten(inputs=conv2)

dense1 = tf.layers.dense(inputs=flat1, units=1024, activation='relu', name='dense_layer_1')

dropout1 = tf.layers.dropout(inputs=dense1, rate=0.4, name='dropout_layer_1')

dense2 = tf.layers.dense(inputs=dropout1, units=10, activation='softmax', name='softmax_output_layer')


# outputs

y_pred = dense2

y_pred_cls = tf.argmax(y_pred, axis=1)

## we do not want gradient to be backpropagated to labels, so we pass it through tf.stop_gradient

label_stopped = tf.stop_gradient(label)

cost_function = tf.nn.softmax_cross_entropy_with_logits_v2(labels=label_stopped, logits=dense2)

cost = tf.reduce_mean(cost_function)

optimizer = tf.train.AdamOptimizer()

train = optimizer.minimize(cost)



correct_prediction  = tf.equal(y_pred_cls, tf.argmax(label, axis=1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [29]:
# training + test loop

## setting number of epoch
num_epoch = 10

# for every epoch
for i in range(num_epoch):
  print('Epoch: ', i + 1)
  
  # init to training data
  sess.run(train_init_op)
  # train all the batches
  print('Train')
  for j in range(num_iterations_per_epoch):
    sess.run(train)
    if j % 200 == 0:
      print('iteration: ', j)
      print('loss: ', sess.run(cost))
      print('accuracy: ', sess.run(accuracy))
    
  
  # init to test data
  sess.run(test_init_op)
  # run on test data and print loss + accuracy
  print('Test')
  print('loss: ', sess.run(cost))
  print('accuracy: ', sess.run(accuracy))

Epoch:  1
Train
iteration:  0
loss:  2.3003616
accuracy:  0.15625
iteration:  200
loss:  1.5562592
accuracy:  0.9375
iteration:  400
loss:  1.5234406
accuracy:  0.90625
iteration:  600
loss:  1.5270953
accuracy:  0.875
iteration:  800
loss:  1.5066153
accuracy:  0.96875
iteration:  1000
loss:  1.4930961
accuracy:  1.0
iteration:  1200
loss:  1.4689977
accuracy:  1.0
iteration:  1400
loss:  1.4699483
accuracy:  1.0
iteration:  1600
loss:  1.4924018
accuracy:  0.9375
Test
loss:  1.4988601
accuracy:  0.963
Epoch:  2
Train
iteration:  0
loss:  1.5226343
accuracy:  0.96875
iteration:  200
loss:  1.4812655
accuracy:  1.0
iteration:  400
loss:  1.4616954
accuracy:  0.96875
iteration:  600
loss:  1.4845674
accuracy:  0.96875
iteration:  800
loss:  1.4906473
accuracy:  0.96875
iteration:  1000
loss:  1.4923426
accuracy:  1.0
iteration:  1200
loss:  1.4941297
accuracy:  0.96875
iteration:  1400
loss:  1.4611504
accuracy:  1.0
iteration:  1600
loss:  1.4924505
accuracy:  1.0
Test
loss:  1.487898


iteration:  400
loss:  1.4870092
accuracy:  1.0
iteration:  600
loss:  1.4913694
accuracy:  1.0
iteration:  800
loss:  1.4924002
accuracy:  0.9375
iteration:  1000
loss:  1.4924002
accuracy:  1.0
iteration:  1200
loss:  1.5549004
accuracy:  0.90625
iteration:  1400
loss:  1.4611502
accuracy:  1.0
iteration:  1600
loss:  1.4924002
accuracy:  1.0
Test
loss:  1.4819025
accuracy:  0.9792
Epoch:  8
Train
iteration:  0
loss:  1.4611505
accuracy:  0.96875
iteration:  200
loss:  1.4611502
accuracy:  1.0
iteration:  400
loss:  1.4611502
accuracy:  0.96875
iteration:  600
loss:  1.4611502
accuracy:  1.0
iteration:  800
loss:  1.4924002
accuracy:  0.96875
iteration:  1000
loss:  1.4924002
accuracy:  1.0
iteration:  1200
loss:  1.5388948
accuracy:  0.90625
iteration:  1400
loss:  1.4611502
accuracy:  1.0
iteration:  1600
loss:  1.494741
accuracy:  1.0
Test
loss:  1.4877033
accuracy:  0.9731
Epoch:  9
Train
iteration:  0
loss:  1.4924002
accuracy:  0.96875
iteration:  200
loss:  1.4611502
accuracy:

In [30]:
# eval
sess.run(val_init_op)
print('loss: ', sess.run(cost))
print('accuracy: ', sess.run(accuracy))

loss:  1.481092
accuracy:  0.98


### Now to figure out how to save the tf model and weights

In [0]:
saver = tf.train.Saver()

In [35]:
saver.save(sess=sess, save_path='./model_files/model.ckpt', )

'./model_files/model.ckpt'

fiddling around with CLI to check if model was saved

In [42]:
!ls

datalab  MNIST_data  model_files


In [43]:
%cd model_files

/content/model_files


In [45]:
!ls -l

total 227596
-rw-r--r-- 1 root root        77 Jul 27 09:30 checkpoint
-rw-r--r-- 1 root root   7838336 Jul 27 09:30 model.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root       985 Jul 27 09:30 model.ckpt.index
-rw-r--r-- 1 root root 225206726 Jul 27 09:30 model.ckpt.meta
